In [1]:
import os 
import torch 
import torch.nn 
import torchvision.models as models
import torchvision.transforms as transforms
import torch.nn.functional as F 
import torchvision.utils as utils
import cv2 
import matplotlib.pyplot as plt
import numpy as np 
import pandas as pd
from PIL import Image
import argparse

In [2]:
horses_images_names = []
bikes_images_names = []

hFolder = "Assignment2_BikeHorses/Horses"
bFolder = "Assignment2_BikeHorses/Bikes"

for image in os.listdir(hFolder):
    if(image.endswith(".jpg")):
        horses_images_names.append(image)
        
print(len(horses_images_names))

for image in os.listdir(bFolder):
    if(image.endswith(".jpg")):
        bikes_images_names.append(image)

print(len(bikes_images_names))

99
80


In [3]:
transform = transforms.Compose([        # Defining a variable transforms
transforms.Resize(256),                # Resize the image to 256×256 pixels
transforms.CenterCrop(224),            # Crop the image to 224×224 pixels about the center
transforms.ToTensor(),                 # Convert the image to PyTorch Tensor data type
transforms.Normalize(                  # Normalize the image
mean=[0.485, 0.456, 0.406],            # Mean and std of image as also used when training the network
std=[0.229, 0.224, 0.225]      
)])

In [4]:
horses_images = []
bikes_images = []

for i in range(len(horses_images_names)):
    img = Image.open(hFolder + "/" + horses_images_names[i])
    transformed_img = transform(img)
    batch_img = torch.unsqueeze(transformed_img, 0)
    horses_images.append(batch_img)    
    
for i in range(len(bikes_images_names)):
    img = Image.open(bFolder + "/" + bikes_images_names[i])
    transformed_img = transform(img)
    batch_img = torch.unsqueeze(transformed_img, 0)
    bikes_images.append(batch_img) 
    
horses_images[0].shape

torch.Size([1, 3, 224, 224])

In [5]:
print(horses_images[0].shape)
print(bikes_images[0].shape)

torch.Size([1, 3, 224, 224])
torch.Size([1, 3, 224, 224])


In [6]:
alexnet = models.alexnet(pretrained = True)
alexnet.eval()

Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to C:\Users\Saikrishna/.cache\torch\hub\checkpoints\alexnet-owt-7be5be79.pth


  0%|          | 0.00/233M [00:00<?, ?B/s]

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [7]:
features = []

In [8]:
preprocessed_images = horses_images
preprocessed_images.extend(bikes_images)

In [9]:
for img in preprocessed_images:
    with torch.no_grad():
        feature = alexnet(img).detach().numpy()
    features.append(feature)

In [10]:
features[0].shape

(1, 1000)

In [11]:
for i in range(len(features)):
    features[i] = features[i].T.reshape((1000,))

In [13]:
features1 = np.array(features)
features1.shape

(179, 1000)

## Train-Test-Split

In [14]:
labels = np.zeros(len(preprocessed_images))
for i in range(len(preprocessed_images)):
     if(i<99):
        labels[i] = 1

print(labels)

[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [18]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(features, labels, train_size=0.8, random_state=42)

## Model Prediction

In [19]:
from sklearn import svm
model = svm.SVC(C=0.0005,kernel='linear', class_weight = 'balanced', gamma = 'scale')
model.fit(x_train,y_train)

SVC(C=0.0005, class_weight='balanced', kernel='linear')

In [20]:
acc = model.score(x_test,y_test)
print(acc)

1.0


In [21]:
from sklearn.linear_model import LogisticRegression
model_lr = LogisticRegression(max_iter = 1000)
model_lr.fit(x_train, y_train)

LogisticRegression(max_iter=1000)

In [22]:
acc1 = model_lr.score(x_test,y_test)
print(acc1)

1.0
